# In this notebook I tried experimenting with Nvidia Retinanet model

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir("./retinanet_nvidia/retinanet-examples/")

In [3]:
from retinanet.model import Model

ImportError: libnvinfer.so.6: cannot open shared object file: No such file or directory

In [2]:
import torch, torchvision
from utils.dataset import *
from utils.evaluator import *
from utils.trainer import MTSDTrainer

# from detectron2.data import DatasetCatalog
# from detectron2.utils.logger import setup_logger

# import some common libraries
import numpy as np
import os, json, cv2, random
import matplotlib.pyplot as plt

# import some common detectron2 utilities
# from detectron2 import model_zoo
# from detectron2.engine import DefaultPredictor
# from detectron2.config import get_cfg
# from detectron2.utils.visualizer import Visualizer
#from detectron2.data import MetadataCatalog, DatasetCatalog
# from detectron2.engine import DefaultTrainer
from datetime import datetime


# from detectron2_backbone import backbone
# from detectron2_backbone.config import add_backbone_config

#import the COCO Evaluator to use the COCO Metrics
# from detectron2.evaluation import COCOEvaluator, inference_on_dataset
# from detectron2.data import build_detection_test_loader


# converting MTSD annotations to json

In [3]:
ann_d="/mnt/CommonData/dataset/filtered_annotations_2/"
img_d="/home/pkos/Desktop/mtsd_fully_annotated/images/"
l_id_f="/home/pkos/workspace/inz/praca_inzynierska/lbl_id_dict.pkl"
id_f_train="/home/pkos/Desktop/mtsd_fully_annotated/splits/train.txt"
id_f_val = "/home/pkos/Desktop/mtsd_fully_annotated/splits/val.txt"

In [4]:
with open('dataset_dicts.pkl', 'rb') as f:
    dataset_dicts = pickle.load(f)

In [5]:
with open(l_id_f, 'rb') as f:
        lbl_id_map = pickle.load(f)
labels = list(lbl_id_map.keys())

In [6]:
img_d = "/data/images/"

In [7]:
dict_coco_train = convert_MTSD_to_COCO(id_f_train, ann_d, img_d, l_id_f)

In [8]:
dict_coco_val = convert_MTSD_to_COCO(id_f_val, ann_d, img_d, l_id_f)

In [10]:
output_json = "/home/pkos/workspace/inz/praca_inzynierska/mtsd_coco_dict_train.json"
write_json(dict_coco_train, output_json)
output_json = "/home/pkos/workspace/inz/praca_inzynierska/mtsd_coco_dict_val.json"
write_json(dict_coco_val, output_json)

# separating train and val

In [5]:
img_d_train="/home/pkos/Desktop/mtsd_fully_annotated/images_train/"
img_d_val = "/home/pkos/Desktop/mtsd_fully_annotated/images_val/"

In [4]:
train = read_list_from_file('/mnt/CommonData/dataset/mtsd_fully_annotated/splits/train.txt')
val = read_list_from_file('/mnt/CommonData/dataset/mtsd_fully_annotated/splits/val.txt')

In [11]:
import os
from shutil import copyfile
for file_id in train:
    src = img_d + file_id + ".jpg"
    dst = img_d_train + file_id +".jpg"
    copyfile(src, dst)
    
for file_id in val:
    src = img_d + file_id + ".jpg"
    dst = img_d_val + file_id +".jpg"
    copyfile(src, dst)
    

# inference

In [23]:
model = torch.load("/home/pkos/workspace/inz/praca_inzynierska/retinanet_nvidia/trained_models/model_20k.pth")

In [26]:
model.keys()

dict_keys(['backbone', 'classes', 'state_dict', 'iteration', 'optimizer', 'scheduler'])

In [20]:
from torchvision.models.resnet import resnet34
model = resnet34(pretrained=True).eval().cuda()

In [22]:
model.load_state_dict(torch.load("/home/pkos/workspace/inz/praca_inzynierska/retinanet_nvidia/trained_models/model_20k.pth")['or'])

RuntimeError: Error(s) in loading state_dict for ResNet:
	Missing key(s) in state_dict: "conv1.weight", "bn1.weight", "bn1.bias", "bn1.running_mean", "bn1.running_var", "layer1.0.conv1.weight", "layer1.0.bn1.weight", "layer1.0.bn1.bias", "layer1.0.bn1.running_mean", "layer1.0.bn1.running_var", "layer1.0.conv2.weight", "layer1.0.bn2.weight", "layer1.0.bn2.bias", "layer1.0.bn2.running_mean", "layer1.0.bn2.running_var", "layer1.1.conv1.weight", "layer1.1.bn1.weight", "layer1.1.bn1.bias", "layer1.1.bn1.running_mean", "layer1.1.bn1.running_var", "layer1.1.conv2.weight", "layer1.1.bn2.weight", "layer1.1.bn2.bias", "layer1.1.bn2.running_mean", "layer1.1.bn2.running_var", "layer1.2.conv1.weight", "layer1.2.bn1.weight", "layer1.2.bn1.bias", "layer1.2.bn1.running_mean", "layer1.2.bn1.running_var", "layer1.2.conv2.weight", "layer1.2.bn2.weight", "layer1.2.bn2.bias", "layer1.2.bn2.running_mean", "layer1.2.bn2.running_var", "layer2.0.conv1.weight", "layer2.0.bn1.weight", "layer2.0.bn1.bias", "layer